In [21]:
"""
STABLE
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
STABLE
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
STABLE
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L::SparseMatrixCSC{Int64,Int64} = laplacian_matrix(g)
    a::Array{Float64,1} = lufact(L) \ (b - mean(b))
    p::Array{Float64,1} = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
STABLE
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end
;

In [22]:
"""
STABLE
soft threshold
"""
soft(c,lambda) = sign(c).*max(abs(c)-lambda/2,0)

"""
STABLE
computes gradient
"""
function gradient2(a,a_0,u,L,rho,b,y)
    convb::Array{Float64,2} =zeros(length(b),1)+b
    grad::Array{Float64,2} = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-convb)
    return grad
end;


function gradient(a,a_0,u,L,rho,b,y)
    grad = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-b)
    return grad
end;



"""
STABLE
computes hessian
"""
function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return hess
end;




"""
STABLE
newton raphson for a update
"""
function newton(y_i,a_0,L,rho,b,u)
    a::Array{Float64,2} = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - inv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
        println(iters)
    end
    if(iters == MAX_ITER)
        print("max iter reached")
    end
    return a
end
;

In [23]:
"""
2 Temp Vars UNSTABLE
ADMM Function 
Parameters:
A - Array of Array{Int64,2} of connections where each column is connections at time t
L - Array of SparseMatrixCSC(Int64,Int64) where each index i is the laplacian at time t= i-1
t - number of ending nodes
t_0 - number of starting nodes
lambda,rho -  Float64s for tuning and optimization
"""
function ADMM(A,L,t,t_0, rho, lambda)
    new = t-t_0
    a = zeros(t-1,new)
    b = zeros(t_0,1) #::Array{Float64,2}
    u = zeros(t-1,new)
    iters = 0
    diff = 1.0
    b_old = b;
	while(diff >STOP_DIFF && iters< MAX_ITER )
        #a update
        for i in 1:new    
            a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:(t_0 +i-1),i])
        end
        #b update
        c = zeros(t-1,1)
        for i in 1:new
            c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ (u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i]))/(rho*new)
        end
        b = soft(c[1:t_0],2*lambda/rho)
        #u update
        for i in 1:new
            u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-[b' zeros(i-1)']')
        end
        diff  = norm(b-b_old)
        b_old = b
        println(diff)
	end
	return b
end
;

In [24]:
using LightGraphs
using DataFrames


data = readtable("training.csv")

t = maximum([maximum(data[:,4]),maximum(data[:,5])])


1461

In [43]:
for i in reverse(1:1)
    print("test")
end

test

In [46]:
g = BinaryTree(0)

for i in 1:t
	add_vertex!(g)
end

for i in 1:size(data,1)
	add_edge!(g,data[i,4],data[i,5])
end
At = adjacency_matrix(g)


A = Array{Int64,2}[]
L =  SparseMatrixCSC{Int64,Int64}[]
Adj =  SparseMatrixCSC{Int64,Int64}[]

new = 1
for i in reverse(1:new)
    n = size(At)[1]
    Ai = zeros(n-i,1)+full(At[n-i,1:n-i+1])
    #Ai[n-i+1] = 0
    push!(A,Ai[1:length(Ai)-1])
    push!(Adj, At[1:n-i+1,1:n-i+1])
end

for adj in Adj
    push!(L,spdiagm(vec(sum(Adj[1],1))).-Adj[1])
end
    
    
t_0 = t-new

LoadError: DimensionMismatch("dimensions must match")

LoadError: BoundsError: attempt to access 0-element Array{SparseMatrixCSC{Int64,Int64},1} at index [1]

In [26]:

const MAX_ITER = 1000
const STOP_DIFF = 0.001;

rho = 1
lambda = 0.0005
a_0 = -5

-5

In [27]:
ADMM(A,L,t,t_0,rho,lambda)

LoadError: BoundsError: attempt to access 1460×1 Array{Float64,2} at index [1:1461]

In [28]:
new = t-t_0
a = ones(t-1,new)
b = ones(t_0,1).-0.5 #::Array{Float64,2}
u = zeros(t-1,new)
iters = 0
diff = 1.0
b_old = b;

In [34]:
new

0

In [33]:
        for i in 1:new    
                print("test")

            a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:(t_0 +i-1),i])
            print("test")
        end

In [85]:
        #b update
        c = zeros(t-1,1)
        for i in 1:new
            c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ (u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i]))/(rho*new)
        end

LoadError: BoundsError: attempt to access 1460×1 Array{Float64,2} at index [1:1461]

In [ ]:
        b = soft(c[1:t_0],2*lambda/rho)
        #u update
        for i in 1:new
            u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-[b' zeros(i-1)']')
        end
        diff  = norm(b-b_old)
        b_old = b
        println(diff)